# Some exploration

In [2]:
import pandas as pd
import numpy as np
from preprocessing import *

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


In [3]:
x_train = pd.read_csv('train_data.csv')
y_train = pd.read_csv('train_labels.csv')
x_test = pd.read_csv('test_data.csv')

# Assign id to each unique questions

In [4]:
x_train, x_test = unique_question_map(x_train, x_test)

In [5]:
x_train.head()

,id,question1,question2,is_duplicate,qid1,qid2
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0,241804
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,1,13483
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,2,241805
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,3,241806
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,4,241807


In [6]:
x_test.head()

,test_id,question1,question2,qid1,qid2
0,15,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,108613,39392
1,20,Why do rockets look white?,Why are rockets and boosters painted white?,449444,314405
2,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,449445,492291
3,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,449446,492292
4,34,What is the best travel website in spain?,What is the best travel website?,449447,275556


In [7]:
df_train = x_train.drop('is_duplicate',1)
df_train = df_train.merge(y_train,on="id")

df_train = df_train[['question1', 'qid1', 'question2', 'qid2','is_duplicate']]
df_train.head()

,question1,qid1,question2,qid2,is_duplicate
0,What is the step by step guide to invest in sh...,0,What is the step by step guide to invest in sh...,241804,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,1,What would happen if the Indian government sto...,13483,0
2,How can I increase the speed of my internet co...,2,How can Internet speed be increased by hacking...,241805,0
3,Why am I mentally very lonely? How can I solve...,3,Find the remainder when [math]23^{24}[/math] i...,241806,0
4,"Which one dissolve in water quikly sugar, salt...",4,Which fish would survive in salt water?,241807,0


In [8]:
print('Total number of question pairs for training: {}'.format(len(df_train)))
print('Duplicate pairs: {}%'.format(round(df_train['is_duplicate'].mean()*100, 2)))
print('Non-duplicate pairs: {}%'.format(100.00 - round(df_train['is_duplicate'].mean()*100, 2)))

print('Number of unique questions: {}'.format(len(pd.concat([df_train['question1'], df_train['question2']]).unique())))

Total number of question pairs for training: 323164
Duplicate pairs: 36.88%
Non-duplicate pairs: 63.12%
Number of unique questions: 449444


In [10]:
pd.concat([df_train['question1'],df_train['question2']]).group_by().count()

AttributeError: 'Series' object has no attribute 'group_by'

,question1,qid1,question2,qid2,is_duplicate
32607,How did playing sports benefit you in real life?,30788,How did playing sports benefit you in real life?,30788,1
37301,Should I stop masturbating?,34942,Should I stop masturbating?,34942,1
56210,How much computer science does an 8 year old U...,51293,How much computer science does an 8 year old U...,51293,1
58083,What are the skills required for big data jobs?,52861,What are the skills required for big data jobs?,52861,1
92052,What are the chances I could be pregnant?,17269,What are the chances I could be pregnant?,17269,0
120431,What does a red exclamation mark mean in Faceb...,102901,What does a red exclamation mark mean in Faceb...,102901,1
150196,Is Indian media worst in the world?,125187,Is Indian media worst in the world?,125187,1
155384,How would you rephrase this sentence?,128973,How would you rephrase this sentence?,128973,0
186894,What is the best way to print a book of photos...,151702,What is the best way to print a book of photos...,151702,1
205054,Are text messages considered legal proof of li...,164359,Are text messages considered legal proof of li...,164359,1


# Pre-processing

In [9]:
df_train = df_train.fillna("")

In [10]:
df_train['question1'] = df_train['question1'].apply(lambda x: preprocess(x), 1)
df_train['question2'] = df_train['question2'].apply(lambda x: preprocess(x), 1)

# Experiment

In [11]:
#https://github.com/ddandur/Quora-DupQ-with-Gensim/blob/master/Quora%20Duplicate%20Questions%20with%20Word2vec%20and%20Keras.ipynb
parameters_dict = {

'documents' : all_docs,
'dm' : 0, # use bag-of-words (dbow) model; 1 uses embedding (dmpv) model
'size' : 200, # size of word/doc vectors
'window' : 15, # # max distance between word and neighbor word for word embeddings
'alpha' : .025, # learning rate - use rate in paper
'min_alpha' : 0.0001, # rate from paper
'min_count' : 5, # ignore words with count less than this
'sample' : 1e-5, # how to configure downsampling for high frequency words
'workers' : cores, # number of cores to use
'hs' : 0, # use negative sampling
'negative' : 5, # used in negative sampling
'dbow_words' : 1, # trains word vectors in addition to document vectors in dbow model
'iter' : 3 # recommended number of epochs is ~20 for dbow model on question comparison   

}

NameError: name 'all_docs' is not defined